In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import json
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
from gezi import tqdm
import gezi

#  所有video的固定静态信息

In [2]:
dis = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/item.parquet')
  dis += [d]
dis += [gezi.read_parquet('../input/eval/item.parquet')]
dis = pd.concat(dis)

100%|██████████| 30/30 [00:01<00:00, 28.45it/s]


In [3]:
dis

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv
0,5893106,25,1411552000,[],600008821,3174145329,2234347078,0,86.110001,0.000,1396.0
1,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.000,1100.0
2,16814925,21,1380506136,[],511097257,3174145329,2234347078,0,317.190002,0.000,4196.0
3,43514419,41,1394444128,[3969330486],3803211341,1574940925,2234347078,0,442.540009,0.000,48.0
4,66011667,64,1330590997,"[359879236, 3688805521, 3190039951]",1009927708,3443952599,3172394293,1112680423,213.639999,0.000,8.0
...,...,...,...,...,...,...,...,...,...,...,...
56531,4204251528,70,1360118075,"[1074701228, 4106478470]",2239251993,1413776458,2234347078,0,771.719971,0.000,56.0
56532,4255628613,33,1413545288,[1134598214],1016220003,1820266297,2234347078,0,35.970001,0.000,480.0
56533,4261704198,32,1382493214,[],1611517696,3174145329,2234347078,0,115.540001,0.000,36.0
56534,4272496705,80,1387029762,"[3309131045, 2561243119, 184102567, 2880021111...",435329116,1820266297,472867291,0,115.540001,0.000,92.0


In [15]:
vinfo = pq.read_table('../input/train/raw.parquet').to_pandas()

In [16]:
vinfo

,vid,image_emb,title,story
0,5893106,"-8.26598072052,1.63619112968,17.9674415588,6.9...","199575,199576,40564",
1,10423101,"-3.63512468338,2.38069868088,14.9096345901,1.2...","717361,318",
2,15819492,"0.107854895294,3.20008993149,20.3623485565,-0....","396,1705,31495,3424,77035,85420,28,17970,9300,...",
3,16814925,"-11.0617599487,1.99434304237,14.9591712952,1.2...","8579,18147,73,227",
4,20360674,"6.6802148819,1.83605599403,15.9008598328,3.244...","22485,118989,12418,396611,61891,3029,26353,396...",
...,...,...,...,...
183788,4276491220,"-4.29484510422,6.66334295273,16.4290809631,-4....","3072,34976,173488,1864,6276,492635,58,19175,24...",
183789,4283547666,"-4.81997919083,13.6302375793,14.3650541306,6.9...","266947,7450,58397,42,58167,44,2577,5",
183790,4283640696,"-0.665687978268,11.5875854492,9.86795520782,3....","406009,91149,52611,66378,290,355096",
183791,4290887658,"2.51159644127,2.58366537094,9.91071224213,4.36...","42,128095,44,188981,28,6705,87381,2679,59,3316...",


In [ ]:
vocab = gezi.Vocab('../input/all/vid.txt')

In [ ]:
vocab.size()

In [ ]:
image_emb = np.random.uniform(-0.05, 0.05,(vocab.size(), 128))

In [ ]:
missing_count = 0
for _, row in tqdm(vinfo.iterrows(), total=len(vinfo), ascii=True):  
  try:
    image_emb[vocab.id(row['vid'])] = np.asarray(list(map(float, row['image_emb'].split(','))))
  except Exception:
    missing_count += 1
#     print(row['image_emb'])
#     break

In [ ]:
missing_count

In [ ]:
np.save('../input/all/image_emb.npy', image_emb)

In [ ]:
dis

In [ ]:
dis[dis.vid==4255628613]

In [ ]:
dis[dis.vid==4283547666]

In [ ]:
dis[dis.vid==5893106]

In [6]:
dis['stars2'] = dis.stars.map(lambda x: ','.join(map(str, x)))

In [ ]:
dis.stars2.values[0]

In [7]:
dis_static = dis.groupby('vid', as_index=False).first()

In [ ]:
dis.info()

In [8]:
dis_static

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,stars2
0,2825,46,1381141499,3191913118,2791670124,607359660,2234347078,0,51.230000,0.0,0.0,3191913118
1,23442,96,1360390566,"[3939310808, 617868672, 1435521369, 2498894837...",4279707160,1820266297,3445748691,0,22.889999,0.0,44.0,"3939310808,617868672,1435521369,2498894837,420..."
2,26537,63,1382702153,"[1849587095, 3965068728, 3208146666, 503441924...",2020729066,1820266297,3172394293,3313280341,102.459999,0.0,268.0,"1849587095,3965068728,3208146666,503441924,345..."
3,37451,83,1401086404,"[3080716205, 2125194400, 2498894837, 243444173...",950308208,1413776458,472867291,0,32.700001,0.0,136.0,"3080716205,2125194400,2498894837,2434441730,37..."
4,40456,67,1376207378,3945148852,47911588,1413776458,2234347078,0,65.400002,0.0,20.0,3945148852
...,...,...,...,...,...,...,...,...,...,...,...,...
178284,4294877210,58,1420379357,"[1951626877, 785794890, 430930810, 1909743963,...",3837363259,1413776458,3172394293,0,143.880005,0.0,1052.0,"1951626877,785794890,430930810,1909743963,3745..."
178285,4294902823,70,1373626174,"[3779642001, 1221620183, 3884752200, 3615377011]",1481880764,1413776458,3172394293,741389469,139.520004,0.0,28.0,"3779642001,1221620183,3884752200,3615377011"
178286,4294922499,69,1427111734,"[4294317133, 892431933, 1853987563, 618480800,...",2957047406,1820266297,3445748691,0,131.889999,0.0,0.0,"4294317133,892431933,1853987563,618480800,1613..."
178287,4294954317,50,1403103478,"[2754170442, 2571785548, 2333623017, 299203150...",4226625658,1820266297,3172394293,0,260.510010,0.0,44.0,"2754170442,2571785548,2333623017,2992031505,12..."


In [17]:
vinfo = vinfo.merge(dis_static, how="left", on="vid")

In [18]:
vinfo = vinfo.fillna(0.)
vinfo.cid = vinfo.cid.astype(int)
vinfo.class_id = vinfo.class_id.astype(int)
vinfo.is_intact = vinfo.is_intact.astype(int)
vinfo.second_class = vinfo.second_class.astype(int)

In [21]:
vinfo

,vid,image_emb,title,story,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,stars2
0,5893106,"-8.26598072052,1.63619112968,17.9674415588,6.9...","199575,199576,40564",,25.0,1.411552e+09,[],600008821,3174145329,2234347078,0,86.110001,0.0,1396.0,
1,10423101,"-3.63512468338,2.38069868088,14.9096345901,1.2...","717361,318",,33.0,1.409657e+09,[],2128734282,3174145329,2234347078,0,981.000000,0.0,436.0,
2,15819492,"0.107854895294,3.20008993149,20.3623485565,-0....","396,1705,31495,3424,77035,85420,28,17970,9300,...",,98.0,1.407204e+09,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.0,1100.0,"2950753864,3600550670,3606342894,3338364112,16..."
3,16814925,"-11.0617599487,1.99434304237,14.9591712952,1.2...","8579,18147,73,227",,21.0,1.380506e+09,[],511097257,3174145329,2234347078,0,317.190002,0.0,4196.0,
4,20360674,"6.6802148819,1.83605599403,15.9008598328,3.244...","22485,118989,12418,396611,61891,3029,26353,396...",,63.0,1.365757e+09,"[792028649, 1936823271]",1552633930,1413776458,2234347078,0,26.160000,0.0,32.0,"792028649,1936823271"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183788,4276491220,"-4.29484510422,6.66334295273,16.4290809631,-4....","3072,34976,173488,1864,6276,492635,58,19175,24...",,81.0,1.382671e+09,3209094375,828839503,1413776458,2234347078,2164216144,456.709991,0.0,0.0,3209094375
183789,4283547666,"-4.81997919083,13.6302375793,14.3650541306,6.9...","266947,7450,58397,42,58167,44,2577,5",,58.0,1.424685e+09,"[2142910698, 1316768238, 1853987563, 1984414365]",2753664459,470292959,2234347078,0,154.779999,0.0,0.0,"2142910698,1316768238,1853987563,1984414365"
183790,4283640696,"-0.665687978268,11.5875854492,9.86795520782,3....","406009,91149,52611,66378,290,355096",,81.0,1.367677e+09,"[2255039294, 3696039767, 1984414365, 193682327...",2553585254,1413776458,3172394293,0,78.480003,0.0,3408.0,"2255039294,3696039767,1984414365,1936823271,10..."
183791,4290887658,"2.51159644127,2.58366537094,9.91071224213,4.36...","42,128095,44,188981,28,6705,87381,2679,59,3316...",,73.0,1.305795e+09,"[2260640674, 2569601900]",201748736,1413776458,3172394293,0,16.350000,0.0,0.0,"2260640674,2569601900"


In [22]:
vinfo.to_csv('../input/all/vinfo_static.csv', index=False)

In [23]:
pd.read_csv('../input/all/vinfo_static.csv')

,vid,image_emb,title,story,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv,stars2
0,5893106,"-8.26598072052,1.63619112968,17.9674415588,6.9...","199575,199576,40564",NaN,25.0,1.411552e+09,[],600008821,3174145329,2234347078,0,86.11,0.0,1396.0,NaN
1,10423101,"-3.63512468338,2.38069868088,14.9096345901,1.2...","717361,318",NaN,33.0,1.409657e+09,[],2128734282,3174145329,2234347078,0,981.00,0.0,436.0,NaN
2,15819492,"0.107854895294,3.20008993149,20.3623485565,-0....","396,1705,31495,3424,77035,85420,28,17970,9300,...",NaN,98.0,1.407204e+09,[2950753864 3600550670 3606342894 3338364112 1...,638675564,1413776458,2234347078,0,6954.20,0.0,1100.0,"2950753864,3600550670,3606342894,3338364112,16..."
3,16814925,"-11.0617599487,1.99434304237,14.9591712952,1.2...","8579,18147,73,227",NaN,21.0,1.380506e+09,[],511097257,3174145329,2234347078,0,317.19,0.0,4196.0,NaN
4,20360674,"6.6802148819,1.83605599403,15.9008598328,3.244...","22485,118989,12418,396611,61891,3029,26353,396...",NaN,63.0,1.365757e+09,[ 792028649 1936823271],1552633930,1413776458,2234347078,0,26.16,0.0,32.0,"792028649,1936823271"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183788,4276491220,"-4.29484510422,6.66334295273,16.4290809631,-4....","3072,34976,173488,1864,6276,492635,58,19175,24...",NaN,81.0,1.382671e+09,3209094375,828839503,1413776458,2234347078,2164216144,456.71,0.0,0.0,3209094375
183789,4283547666,"-4.81997919083,13.6302375793,14.3650541306,6.9...","266947,7450,58397,42,58167,44,2577,5",NaN,58.0,1.424685e+09,[2142910698 1316768238 1853987563 1984414365],2753664459,470292959,2234347078,0,154.78,0.0,0.0,"2142910698,1316768238,1853987563,1984414365"
183790,4283640696,"-0.665687978268,11.5875854492,9.86795520782,3....","406009,91149,52611,66378,290,355096",NaN,81.0,1.367677e+09,[2255039294 3696039767 1984414365 1936823271 1...,2553585254,1413776458,3172394293,0,78.48,0.0,3408.0,"2255039294,3696039767,1984414365,1936823271,10..."
183791,4290887658,"2.51159644127,2.58366537094,9.91071224213,4.36...","42,128095,44,188981,28,6705,87381,2679,59,3316...",NaN,73.0,1.305795e+09,[2260640674 2569601900],201748736,1413776458,3172394293,0,16.35,0.0,0.0,"2260640674,2569601900"
